In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .appName("ETL to Star Schema") \
    .config("spark.jars", "/home/jovyan/jars/postgresql-42.7.3.jar") \
    .getOrCreate()


In [3]:
jdbc_url = "jdbc:postgresql://bigdata_postgres:5432/pet_sales"
connection_properties = {
    "user": "lab_user",
    "password": "lab_password",
    "driver": "org.postgresql.Driver"
}


In [4]:
df = spark.read.jdbc(
        url=jdbc_url,
        table="mock_data",
        properties=connection_properties
    )

df = df.withColumn("sale_id", F.monotonically_increasing_id())

In [5]:
dim_customer = df.groupBy("sale_customer_id").agg(
        F.first("customer_first_name").alias("customer_first_name"),
        F.first("customer_last_name").alias("customer_last_name"),
        F.first("customer_age").alias("customer_age"),
        F.first("customer_email").alias("customer_email"),
        F.first("customer_country").alias("customer_country"),
        F.first("customer_postal_code").alias("customer_postal_code"),
        F.first("customer_pet_type").alias("customer_pet_type"),
        F.first("customer_pet_name").alias("customer_pet_name"),
        F.first("customer_pet_breed").alias("customer_pet_breed")
    ).withColumnRenamed("sale_customer_id", "customer_id")

In [6]:
dim_customer.show(n=3, truncate=False)

+-----------+-------------------+------------------+------------+---------------------+----------------+--------------------+-----------------+-----------------+------------------+
|customer_id|customer_first_name|customer_last_name|customer_age|customer_email       |customer_country|customer_postal_code|customer_pet_type|customer_pet_name|customer_pet_breed|
+-----------+-------------------+------------------+------------+---------------------+----------------+--------------------+-----------------+-----------------+------------------+
|1          |Barron             |Rawlyns           |61          |bmassingham0@army.mil|China           |NULL                |cat              |Priscella        |Labrador Retriever|
|2          |Ham                |Knowller          |78          |cscudder1@time.com   |Poland          |73-115              |bird             |Dalenna          |Labrador Retriever|
|3          |Farleigh           |Langley           |71          |vhuxter2@fotki.com   |Poland  

In [7]:
dim_product = df.groupBy("sale_product_id").agg(
        F.first("product_name").alias("product_name"),
        F.first("product_category").alias("product_category"),
        F.first("product_price").alias("product_price"),
        F.first("product_weight").alias("product_weight"),
        F.first("product_color").alias("product_color"),
        F.first("product_size").alias("product_size"),
        F.first("product_brand").alias("product_brand"),
        F.first("product_material").alias("product_material"),
        F.first("product_description").alias("product_description"),
        F.first("product_rating").alias("product_rating"),
        F.first("product_reviews").alias("product_reviews"),
        F.first("product_release_date").alias("product_release_date"),
        F.first("product_expiry_date").alias("product_expiry_date")
    ).withColumnRenamed("sale_product_id", "product_id")

In [8]:
dim_product.show(n=3, truncate=False)

+----------+------------+----------------+---------------------+---------------------+-------------+------------+-------------+----------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+---------------+--------------------+-------------------+
|product_id|product_name|product_category|product_price        |product_weight       |product_color|product_size|product_brand|product_material|product_description                                                                                                                                       

In [9]:
dim_time = df.select(
        F.col("sale_date").alias("date")
    ).distinct().withColumn("time_id", F.monotonically_increasing_id())

dim_time = dim_time.withColumn("year", F.year("date")).withColumn("month", F.month("date")).withColumn("day", F.dayofmonth("date")).withColumn("quarter", F.quarter("date")).withColumn("dow", F.dayofweek("date"))


In [10]:
dim_time.show(n=3, truncate=False)

+----------+-------+----+-----+---+-------+---+
|date      |time_id|year|month|day|quarter|dow|
+----------+-------+----+-----+---+-------+---+
|2021-11-13|0      |2021|11   |13 |4      |7  |
|2021-12-18|1      |2021|12   |18 |4      |7  |
|2021-08-27|2      |2021|8    |27 |3      |6  |
+----------+-------+----+-----+---+-------+---+
only showing top 3 rows



In [11]:
dim_store = df.groupBy("store_name", "store_location").agg(
        F.first("store_city").alias("store_city"),
        F.first("store_state").alias("store_state"),
        F.first("store_country").alias("store_country"),
        F.first("store_phone").alias("store_phone"),
        F.first("store_email").alias("store_email")
    ).withColumn("store_id", F.monotonically_increasing_id())


In [12]:
dim_store.show(n=3, truncate=False)

+----------+--------------+-----------+-----------+-------------+------------+-----------------------+--------+
|store_name|store_location|store_city |store_state|store_country|store_phone |store_email            |store_id|
+----------+--------------+-----------+-----------+-------------+------------+-----------------------+--------+
|Abata     |11th Floor    |Ibiúna     |NULL       |Montenegro   |660-676-5491|cchevisp5@npr.org      |0       |
|Abata     |12th Floor    |San Vicente|NULL       |Argentina    |230-952-8959|lgoodacregs@vinaora.com|1       |
|Abata     |14th Floor    |Norsborg   |AB         |Portugal     |155-350-7386|shanshaw8h@sfgate.com  |2       |
+----------+--------------+-----------+-----------+-------------+------------+-----------------------+--------+
only showing top 3 rows



In [13]:
dim_supplier = df.groupBy("supplier_name", "supplier_country").agg(
        F.first("supplier_contact").alias("supplier_contact"),
        F.first("supplier_email").alias("supplier_email"),
        F.first("supplier_phone").alias("supplier_phone"),
        F.first("supplier_address").alias("supplier_address"),
        F.first("supplier_city").alias("supplier_city")
    ).withColumn("supplier_id", F.monotonically_increasing_id())


In [14]:
dim_supplier.show(n=3, truncate=False)

+-------------+----------------+----------------+-------------------------+--------------+----------------+------------------+-----------+
|supplier_name|supplier_country|supplier_contact|supplier_email           |supplier_phone|supplier_address|supplier_city     |supplier_id|
+-------------+----------------+----------------+-------------------------+--------------+----------------+------------------+-----------+
|Abata        |Armenia         |Farris Blanking |fblankingd8@google.com.au|530-854-6864  |Suite 51        |Jindong           |0          |
|Abata        |Australia       |Bernadene Bossel|bbossellv@fema.gov       |812-276-2743  |PO Box 14734    |Evansville        |1          |
|Abata        |Brazil          |Vivianna Niblock|vniblockn7@va.gov        |268-303-3219  |Room 816        |Ambohitrolomahitsy|2          |
+-------------+----------------+----------------+-------------------------+--------------+----------------+------------------+-----------+
only showing top 3 rows



In [15]:
fact_sales = df.alias("f").join(dim_customer.alias("c"),F.col("f.sale_customer_id") == F.col("c.customer_id"), "left").join(dim_product.alias("p"),F.col("f.sale_product_id") == F.col("p.product_id"), "left").join(dim_store.alias("s"),
              (F.col("f.store_name") == F.col("s.store_name")) & 
              (F.col("f.store_location") == F.col("s.store_location")), "left") \
        .join(dim_supplier.alias("sup"),
              (F.col("f.supplier_name") == F.col("sup.supplier_name")) & 
              (F.col("f.supplier_country") == F.col("sup.supplier_country")), "left") \
        .join(dim_time.alias("t"),
              F.col("f.sale_date") == F.col("t.date"), "left") \
        .select(
            F.col("f.sale_id").alias("sale_id"),
            F.col("c.customer_id"),
            F.col("p.product_id"),
            F.coalesce(F.col("s.store_id"), F.lit(-1)).alias("store_id"),
            F.coalesce(F.col("sup.supplier_id"), F.lit(-1)).alias("supplier_id"),
            F.col("t.time_id"),
            F.col("sale_quantity").alias("quantity"),
            F.col("sale_total_price").alias("total_price")
        )


In [16]:
fact_sales.show(n=3, truncate=False)

+-------+-----------+----------+--------+-----------+-------+--------+----------------------+
|sale_id|customer_id|product_id|store_id|supplier_id|time_id|quantity|total_price           |
+-------+-----------+----------+--------+-----------+-------+--------+----------------------+
|0      |1          |1         |4263    |785        |242    |4       |487.700000000000000000|
|2      |3          |3         |2986    |2521       |61     |9       |144.240000000000000000|
|3      |4          |4         |2239    |891        |218    |4       |441.090000000000000000|
+-------+-----------+----------+--------+-----------+-------+--------+----------------------+
only showing top 3 rows



In [17]:
dim_customer.write.jdbc(url=jdbc_url, table="dim_customer", mode="overwrite", properties=connection_properties) 
dim_product.write.jdbc(url=jdbc_url, table="dim_product", mode="overwrite", properties=connection_properties)   
dim_store.write.jdbc(url=jdbc_url, table="dim_store", mode="overwrite", properties=connection_properties)    
dim_supplier.write.jdbc(url=jdbc_url, table="dim_supplier", mode="overwrite", properties=connection_properties)   
dim_time.write.jdbc(url=jdbc_url, table="dim_time", mode="overwrite", properties=connection_properties)  
fact_sales.write.jdbc(url=jdbc_url, table="fact_sales", mode="overwrite", properties=connection_properties)

In [18]:
spark.stop()